In [191]:
import sys
import random
import numpy as np
from numpy import array,zeros,dot
from numpy import genfromtxt

In [ ]:
data = genfromtxt('mnist_train.csv', delimiter=',')

In [7]:
a = array([[1,2,3,4,5],[11,12,13,14,15],[11,121,131,141,151]])

In [204]:
class NN:
    def __init__(self, inp_unit, out_unit,  hidden_shape, activation, itr=1000, alpha=0.01, momentum=0.9, decay=0.1):
        self.inp_unit,self.out_unit,self.hidden_shape,self.itr,self.alpha,self.momentum,self.decay,self.activation = inp_unit,\
                                                            out_unit, hidden_shape, itr, alpha, momentum, decay, activation 
        self.flag,self.W,self.B,self.arc_dim = True,0,0,0     
        self.pred,self.h_layer,self.h_layer_A,self.temp,self.Frist_time,self._v,self._m = 0,0,0,0,0,0,0
        self.testing = []
    
    def initz(self,inp_unit,out_unit):
        self.W,self.B,self.arc_dim = 0,0,0
        self.pred,self.temp,self.Frist_time,self._v,self._m = 0,0,True,zeros((len(self.hidden_shape)+1))\
                                                                            ,zeros((len(self.hidden_shape)+1))
        if len(inp_unit.shape) > 1 and len(out_unit.shape) > 1:
            self.W,self.B,self.arc_dim = [],[],[inp_unit.shape[1]] + self.hidden_shape + [out_unit.shape[1]]
        else:
            self.flag = False
    
    def sigmoid(self,x,flag):
        if flag is 1:
            self.testing.append(x)
            x_dash = np.exp(-x)/((1+np.exp(-x))**2)
            self.testing.append(x_dash)
            
        return 1 / (1.0 + np.exp(-x)) if flag == 0 else np.exp(-x)/((1+np.exp(-x))**2)

    def tanh(self,x,flag):
        return np.tanh(x) if flag == 0 else 1.0 - np.tanh(x)**2.0 

    def random_weight(self,shp):
        return np.random.normal(loc = 0, scale = 0.3, size = shp)
        
    def _feed_forward(self,A):
        self.h_layer,self.h_layer_A = [],[]
        if self.Frist_time is True:
            for idx in range(len(self.arc_dim)-1):
                self.W.append(self.random_weight([self.arc_dim[idx],self.arc_dim[idx+1]]))
                self.B.append(self.random_weight([1,self.arc_dim[idx+1]]))
            self.Frist_time = False
        for idx in range(len(self.arc_dim)-2):
            if self.activation[idx] is "sigmoid":
                A = dot(A,self.W[idx]) + self.B[idx]
                self.h_layer.append(A)
                C = self.sigmoid(A,0)
                self.h_layer_A.append(C)
            else:
                A = dot(A,self.W[idx]) + self.B[idx]
                self.h_layer.append(A)
                C = self.tanh(A,0)
                self.h_layer_A.append(C)
        
        if self.activation[-1] is "sigmoid":
            self.temp = dot(C,self.W[idx+1]) + self.B[idx+1]
            self.pred = self.sigmoid(self.temp,0) 
        else:
            self.temp = dot(C,self.W[idx+1]) + self.B[idx+1]
            self.pred = self.tanh(self.temp,0)
    
    def cost_func(self):
        return 0.5*sum((self.out_unit-self.pred)**2)
    
    def _back_prop(self,Inp):
        weight_delta = []
        delta = np.multiply(-(self.out_unit-self.pred), self.sigmoid(self.temp,1))
        weight_delta.append(np.dot(self.h_layer_A[-1].T, delta))
        for idx in reversed(range(len(self.h_layer))):
            delta2 = np.dot(delta, self.W[idx+1].T)*self.sigmoid(self.h_layer[idx],1)
            if idx > 0:
                weight_delta.append(np.dot(self.h_layer_A[idx-1].T, delta2))
            else:
                weight_delta.append(np.dot(Inp.T, delta2))
            delta = delta2        
        weight_delta.reverse()
        return weight_delta
    
    def _weight_update_SG(self,dx,alpha):
        for i in range(len(dx)):
            self.W[i] += alpha * dx[i]
    
    def _weight_update_Momentum(self,dx,alpha,mu):
        for i in range(len(dx)):
            self._v[i] = mu * self._v[i] - alpha * dx[i]
            self.W[i] += self._v[i]
            
    def _weight_update_Nag(self,dx,alpha,mu):
        for i in range(len(dx)):
            v_prev = self._v[i]
            self._v[i] = mu * self._v[i] - alpha * dx[i]
            self.W[i] += -mu * v_prev + (1 + mu) * self._v[i]
            
    def _weight_update_Adam(self,dx,t,alpha,eps=1e-8,beta1=0.9,beta2=0.999):
        for i in range(len(dx)):
            self._m[i] = beta1*self._m[i] + (1-beta1)*dx[i]
            mt = self._m[i] / (1-beta1**t)
            self._v[i] = beta2*self._v[i] + (1-beta2)*(dx[i]**2)
            vt = self._v[i] / (1-beta2**t)
            self.W[i] += - alpha * mt / (np.sqrt(vt) + eps)
            
    def train(self):
        self.initz(self.inp_unit,self.out_unit)
        for i in range(1):
            self._feed_forward(self.inp_unit)
            dx = self._back_prop(self.inp_unit)
            
#             print(dx)
            
#             for j in range(len(dx)):
#                 print(self.W[j].shape,dx[j].shape)
                
#             self._weight_update_SG(dx,self.alpha)
#             if(i%10 == 0):
#                 print(self.cost_func())

In [205]:
# nn = NN(a,array([[1,0],[0,1],[1,0]]),[3,4,3],['tanh','sigmoid','tanh','sigmoid'])
nn = NN(train_x[:100],train_y_1[:100],[10,15,20],['tanh','tanh','tanh','tanh'])
nn.train()
# nn.W


C:\Users\vedan\AppData\Local\conda\conda\envs\dlwin36\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp
C:\Users\vedan\AppData\Local\conda\conda\envs\dlwin36\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in square
C:\Users\vedan\AppData\Local\conda\conda\envs\dlwin36\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
C:\Users\vedan\AppData\Local\conda\conda\envs\dlwin36\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp
C:\Users\vedan\AppData\Local\conda\conda\envs\dlwin36\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in square
C:\Users\vedan\AppData\Local\conda\conda\envs\dlwin36\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [211]:
# nn = NN(a,array([[1,0],[0,1],[1,0]]),[3,4,5],['tanh','sigmoid','tanh','sigmoid'])
# a1.reverse()
# a1
nn.testing[6]

array([[  3.54204077e+01,  -2.83090683e+02,  -1.34699750e+03,
         -1.33735528e+03,   3.49191947e+02,  -7.67068858e+01,
         -1.94510770e+02,   6.01576308e+02,   6.50347703e+02,
          1.62384447e+02],
       [  6.81001403e+02,  -9.45645315e+02,  -1.41834785e+03,
         -3.79908409e+02,   2.17235690e+02,   6.43488481e+01,
         -2.07474791e+02,  -4.86114957e+02,   1.47713765e+02,
         -1.39579635e+02],
       [  5.36980873e+02,  -1.05353155e+03,  -2.91970130e+02,
         -3.13908462e+02,  -4.33307998e+02,  -5.31411273e+02,
         -6.09999611e+02,   6.03285830e+02,  -6.60182402e+01,
          5.23354977e+02],
       [  1.07830055e+03,   3.44838212e+02,  -7.75973317e+02,
         -2.62688200e+02,   5.34632653e+02,  -8.24827064e+00,
         -6.59137416e+01,   3.66626504e+02,  -3.86963438e+02,
         -1.55786230e+02],
       [  2.97737529e+01,   4.52811355e+02,  -1.14361967e+03,
          1.45402931e+02,   1.13359778e+02,  -1.02896415e+03,
         -4.71047940e+02

In [212]:
nn.testing[7]

array([[  4.14106843e-016,   1.13573938e-123,               nan,
                      nan,   2.22773783e-152,   4.85984889e-034,
          3.35000694e-085,   5.47934847e-262,   3.61061422e-283,
          3.00144611e-071],
       [  1.75731058e-296,               nan,               nan,
          0.00000000e+000,   4.52624811e-095,   1.13148901e-028,
          7.84952218e-091,   0.00000000e+000,   7.05873612e-065,
          2.40621695e-061],
       [  6.19683429e-234,               nan,   1.58118809e-127,
          4.69123573e-137,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   9.91500259e-263,   2.13129005e-029,
          5.12650499e-228],
       [  0.00000000e+000,   1.73247689e-150,               nan,
          8.24070431e-115,   6.48618131e-233,   2.61573827e-004,
          2.36606004e-029,   5.97217186e-160,   0.00000000e+000,
          2.20242011e-068],
       [  1.17333879e-013,   2.22088968e-197,               nan,
          7.11720755e-064,   5.86778339e-05

In [168]:
(1e-10)*10

1e-09

In [88]:
list.reverse(a)

In [69]:
print(nn.W[0].shape,
nn.W[1].shape,
nn.W[2].shape,
nn.W[6].shape)

IndexError: list index out of range

In [3]:
b = array([[1,0],[0,1],[1,0]])

In [106]:
q = array([[1,2,3],[4,5,6]])
q = q + array([1])

# Preprocessing

In [3]:
train = data[:50000]
valid = data[50000:]

In [41]:
train_y = train[:,0]
train_x = train[:,1:]
train_y_1 = np.eye(10)[train_y.astype(int)]
valid_y = valid[:,0]
valid_x = valid[:,1:]
valid_y_1 = np.eye(10)[valid_y.astype(int)]

# Testing Tensorflow

In [133]:
import tensorflow as tf

In [163]:
x = tf.placeholder(tf.float32, [None, 5])
y = tf.placeholder(tf.float32, [None, 2])
w1 = tf.placeholder(tf.float32, [5, 3])
w2 = tf.placeholder(tf.float32, [3, 4])
w3 = tf.placeholder(tf.float32, [4, 5])
w4 = tf.placeholder(tf.float32, [5, 2])
b1 = tf.placeholder(tf.float32, [1, 3])
b2 = tf.placeholder(tf.float32, [1, 4])
b3 = tf.placeholder(tf.float32, [1, 5])
b4 = tf.placeholder(tf.float32, [1, 2])

In [164]:
g1 = tf.tanh(tf.add(tf.matmul(x, w1), b1))
print(g1)
g2 = tf.sigmoid(tf.add(tf.matmul(g1, w2), b2))
print(g2)
g3 = tf.tanh(tf.add(tf.matmul(g2, w3), b3))
print(g3)
g4 = (tf.add(tf.matmul(g3, w4), b4))
print(g4)

Tensor("Tanh_17:0", shape=(?, 3), dtype=float32)
Tensor("Sigmoid_9:0", shape=(?, 4), dtype=float32)
Tensor("Tanh_18:0", shape=(?, 5), dtype=float32)
Tensor("Add_35:0", shape=(?, 2), dtype=float32)


In [165]:
with tf.Session() as sess:
    f = sess.run(g4,feed_dict = {x: a, y: b, w1: nn.W[0], w2: nn.W[1], w3: nn.W[2], w4: nn.W[3],
                             b1: nn.B[0], b2: nn.B[1], b3: nn.B[2], b4: nn.B[3]})

In [166]:
f

array([[-0.12157428, -0.07123873],
       [-0.11776307, -0.07139242],
       [-0.13612098, -0.07489069]], dtype=float32)

In [8]:
array([[1,2],[2,3]]) - array([[3,1],[1,2]])

array([[-2,  1],
       [ 1,  1]])